In [ ]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"


<img src='../../../common/logo_DH.png' align='left' width=35%/>

### Checkpoint: Clasificacion de texto

Trabajaremos con un dataset de noticias (en ingles) sobre diferentes temas: world, sports, business, sci/tech.

La idea es vectorizar el corpus de noticias e implementar un clasificador para identificar a qué tema pertenecen las noticias.

Trabajaremos con una versión reducida de un corpus que pueden descargar completo (~30Mb) del siguiente sitio:

https://github.com/mhjabreel/CharCnn_Keras

#### Ejercicio 1

Generamos el corpus:

Importen los datos con pandas. Los datos se encuentran en '../Data/ag_news_reduced.csv'. 

El dataset contiene cuatro columnas: la primera tiene un entero que indica a qué clase pertenece la noticia. La segunda es el nombre de la clase, la tercera es el título y la cuarta es una descripción de la noticia.

Como no tenemos las noticias enteras sino solo un resumen, conviene que generemos el corpus concatenando el título y la descripción en un sólo texto. 

Generen una columna en el dataframe que sea la concatenación del título y la descripción. No olviden agregar un espacio en blanco para no pegar la última palabra del título con la primera de la descrición.

¿Cuántos artículos hay de cada clase?

In [ ]:
import pandas as pd

data=pd.read_csv('../Data/ag_news_reduced.csv');

data.head()

In [ ]:
data['news']=data['title']+ ' '+data['description'];

In [ ]:
data['news'][0]

In [ ]:
data['class_name'].value_counts()

In [ ]:
len(data)

#### Ejercicio 2

¿Cuáles son las palabras más frecuentes dentro de cada clase?

Ayuda: Pueden vectorizar el corpus (dividido por temas) usando CountVectorizer() y luego sumar las filas de la matriz para obtener el número total de veces que aparece cada término.

Ayuda2: No olviden remover stopwords al vectorizar!

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
import numpy as np
stop_words=stopwords.words('english');

vectorizer=CountVectorizer(stop_words=stop_words);

clases=['World','Sports','Business','Sci/Tech'];

for clase in range(1,5):
    X=vectorizer.fit_transform(data[data['class']==clase]['news']);
    counts=X.sum(axis=0);
    counts=np.array(counts);
    
    indices=np.argsort(counts);
    valores=np.sort(counts);
    indices=indices[0][::-1];
    valores=valores[0][::-1];
    terms=np.array(vectorizer.get_feature_names());

    print('\n Clase ',clases[clase-1])
    print(terms[indices[:20]])

#### Ejercicio 3

Vectorizar el corpus usando TfidfVectorizer y entrenar un clasificador de tipo Multinomial Naive Bayes (MultinomialNB). Para empezar, hagan un simple train_test_split de los datos y vean la performance (accuracy) en el set de validacion.

Luego vean si pueden mejorar esa performance optimizando el parámetro alpha del modelo haciendo una Gridsearch cross validation.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV,StratifiedKFold,train_test_split
from sklearn.metrics import accuracy_score

train,test=train_test_split(data,stratify=data['class'],random_state=3);

vectorizer=TfidfVectorizer();
X_train=vectorizer.fit_transform(train['news']);
y_train=train['class'];

X_test=vectorizer.transform(test['news']);
y_test=test['class'];

nbc=MultinomialNB();

nbc.fit(X_train,y_train);
y_pred=nbc.predict(X_test);

print('Accuracy:',accuracy_score(y_test,y_pred))



In [ ]:
# Optimizamos en alpha

vectorizer=TfidfVectorizer();

X=vectorizer.fit_transform(data['news']);
y=data['class'];

skf=StratifiedKFold(n_splits=3,random_state=3,shuffle=True);

params={'alpha':np.arange(0.01,1,0.05)};
GS_CV=GridSearchCV(MultinomialNB(),params,cv=skf,verbose=1,n_jobs=3,iid=False);
GS_CV.fit(X,y);
print('best score:',GS_CV.best_score_)
print('best params:',GS_CV.best_params_)

In [ ]:
X.shape

#### Ejercicio 4

Repetir lo anterior removiendo stopwords al vectorizar.

¿Cambia la performance?  ¿Cuánto se redujo la dimensionalidad?

In [ ]:
# Include Stopwords
from nltk.corpus import stopwords
stop_words=stopwords.words('english');
stop_words.append('39');
stop_words.append('reuters');

vectorizer=TfidfVectorizer(stop_words=stop_words);

X_train=vectorizer.fit_transform(data['news']);
y_train=data['class'];

skf=StratifiedKFold(n_splits=3,random_state=0,shuffle=True);
params={'alpha':np.arange(0.01,1,0.05)};
GS_CV=GridSearchCV(MultinomialNB(),params,cv=skf,verbose=1,n_jobs=3,iid=False);
GS_CV.fit(X_train,y_train);
print('best score:',GS_CV.best_score_)
print('best params:',GS_CV.best_params_)


In [ ]:
print(X_train.shape)

#### Ejercicio 5

Utilizar los parámetros min_df y max_df para remover términos que aparecen en muy pocos documentos o que aparecen en demasiados (stopwords específicas del corpus).

Reentrenar el modelo y ver si cambia la performance.



In [ ]:
# min_df

vectorizer=TfidfVectorizer(stop_words=stop_words,min_df=2,max_df=0.8);
X_train=vectorizer.fit_transform(data['news']);
y_train=data['class'];

print(X_train.shape)

skf=StratifiedKFold(n_splits=3,random_state=0,shuffle=True);
params={'alpha':np.arange(0.1,1,0.05)};
GS_CV=GridSearchCV(MultinomialNB(),params,cv=skf,verbose=1,n_jobs=3,iid=False);
GS_CV.fit(X_train,y_train);
print('best score:',GS_CV.best_score_)
print('best params:',GS_CV.best_params_)


#### Ejercicio 6

Repetir el análisis incluyendo bigramas en la vectorizacion.

In [ ]:
# n-grams

vectorizer=TfidfVectorizer(stop_words=stop_words,min_df=2,ngram_range=(1,2));

X_train=vectorizer.fit_transform(data['news']);
y_train=data['class'];

print(X_train.shape)

skf=StratifiedKFold(n_splits=3,random_state=0,shuffle=True);
params={'alpha':np.arange(0.05,0.15,0.01)};
GS_CV=GridSearchCV(MultinomialNB(),params,cv=skf,verbose=1,n_jobs=3,iid=False);
GS_CV.fit(X_train,y_train);
print('best score:',GS_CV.best_score_)
print('best params:',GS_CV.best_params_)